# Import

In [1]:
import numpy as np
import random as rd

# Define

In [8]:
class Sigmoid():
    def trans(self, x):
        out_put = np.zeros([len(x)])
        for i in range(len(x)):
            out_put[i] = 1 / (1 + np.exp(-x[i]))
        
        return out_put
    
    def diff(self, x):
        out_put = np.zeros([len(x), len(x)])
        for i in range(len(x)):
            out_put[i][i] = 1 / (1 + np.exp(x[i])*(1 + np.exp(-x[i])))
            
        return out_put
    
class Identity():
    def trans(self, x):
        return x
    
    def diff(self, x):
        return np.identity(len(x))

class Layer():
    def __init__(self, layers_from, nodes_n, active_function):
        self.lf = layers_from # layers connected before
        self.lt = [] # layers connected after
        self.nodes_n = nodes_n
        self.nodes_before = np.zeros([nodes_n])
        self.nodes_after = np.zeros([nodes_n])
        self.nodes_diff = np.zeros([nodes_n])
        self.af = active_function
    
    def forward(self):
        self.nodes_after = self.af.trans(self.nodes_before)
    
    def jacobi(self):
        self.nodes_diff = self.af.diff(self.nodes_before)
    
class DogikoNN():
    def __init__(self):
        self.training_data = None
        self.pp_linear = {
            "multi" : None,
            "trans" : None
        } # pre-processing linear translation
        self.layers = []
    
    def set_data(self, input_data, input_answer, input_weights = None):
        # type input_data : numpy array
        # type input_answer : numpy array
        self.t_data = input_data
        self.t_answer = input_answer
        self.t_weights = input_weights
        
    def define_normalized(normalized_algorithm):
        self.normal_alg = normalized_algorithm
        
    def insert_layer(layer):
        self.layers.append(layer)
        
    def build():
        